# **NFL Big Data Bowl 2026 - Análisis de Movimiento de Jugadores**

Este notebook forma parte de mi participación en el NFL Big Data Bowl 2026, una competición de analítica deportiva organizada por la NFL







---


## **Estructura de los Datos**
El NFL Big Data Bowl 2026 proporciona datos de Next Gen Stats (NGS) capturados a 10 Hz (10 frames por segundo). Los datos están divididos en dos fases temporales: antes y después del pase.

<br/>

### **Archivos de Datos Principales**

* **input_2023_w[01-18].csv** - Tracking ANTES del Pase: contiene datos de tracking desde el snap hasta el momento en que el QB suelta el balón. Hay 18 archivos, uno por cada semana de la temporada 2023.

  * Clave Primaria Compuesta: (game_id, play_id, nfl_id, frame_id)
  * Sin claves foráneas (archivo integrado)

  * Variables de identificación:

      - game_id: Identificador único del partido
      - play_id: Identificador de la jugada (no único entre partidos)
      - nfl_id: Identificador único del jugador
      - frame_id: Frame temporal (inicia en 1 para cada jugada)

  * Variables de tracking:

      - x, y: Coordenadas del jugador (0-120 yardas x 0-53.3 yardas)
      - s: Velocidad (yardas/segundo)
      - a: Aceleración (yardas/segundo²)
      - o: Orientación del cuerpo del jugador (grados)
      - dir: Dirección del movimiento (grados)

  * Variables del jugador (integradas):

      - player_name: Nombre del jugador
      - player_height: Altura (ft-in)
      - player_weight: Peso (lbs)
      - player_birth_date: Fecha de nacimiento (yyyy-mm-dd)
      - player_position: Posición del jugador
      - player_side: Lado del equipo (Offense/Defense)
      - player_role: Rol en la jugada (Passer, Targeted Receiver, Defensive Coverage, Other Route Runner)

  * Variables de contexto:

      - play_direction: Dirección de avance de la ofensiva (left/right)
      - absolute_yardline_number: Distancia desde la end zone
      - ball_land_x, ball_land_y: Posición donde aterrizará el balón
      - num_frames_output: Número de frames a analizar en el archivo output
      - player_to_predict: Indica si este jugador será incluido en predicciones (bool)

<br/>

* **output_2023_w[01-18].csv** - Tracking DESPUÉS del Pase. Contiene datos de tracking desde que el QB suelta el balón hasta que la jugada termina (recepción, pase incompleto, intercepción). 18 archivos semanales.


  * Clave Primaria Compuesta: (game_id, play_id, nfl_id, frame_id)

  * Relaciona con: input_2023_w[N].csv mediante (game_id, play_id, nfl_id)

  * Variables:

    - game_id, play_id, nfl_id, frame_id:
    - x, y: Coordenadas del jugador durante el vuelo del balón

    <br/>

**Nota importante: El frame_id en output inicia nuevamente en 1. El número máximo de frames para cada jugador coincide con num_frames_output del archivo input correspondiente.**

<br/>

* **supplementary.csv** - Información Contextual de Juego/Jugada. Archivo único con metadata completa sobre partidos y jugadas. Combina información que normalmente estaría en archivos separados de games/plays.


  * Clave Primaria Compuesta: (game_id, play_id)
  * Sin claves foráneas (archivo de referencia)
  * season: Temporada
      - week: Semana del juego
      - game_date: Fecha (mm/dd/yyyy)
      - game_time_eastern: Hora de inicio (HH:MM:SS, EST)
      - home_team_abbr, visitor_team_abbr: Códigos de equipos
      - home_final_score, visitor_final_score: Puntajes finales
      - pre_snap_home_score, pre_snap_visitor_score: Puntajes antes del snap

  * Variables de la jugada:
      - play_description: Descripción textual completa
      - quarter: Cuarto del juego
      - game_clock: Tiempo en el reloj (MM:SS)
      - down: Down actual (1-4)
      - yards_to_go: Yardas para primer down
      - possession_team: Equipo con posesión
      - defensive_team: Equipo en defensa
      - yardline_side, yardline_number: Línea de scrimmage

  * Variables del pase:

      - pass_result: Resultado (C=completo, I=incompleto, S=sack, IN=intercepción, R=scramble)
      - pass_length: Distancia del pase más allá de la LOS (negativo si es hacia atrás)
      - offense_formation: Formación ofensiva
      - receiver_alignment: Alineación de receptores (ej: 2x2, 3x1)
      - route_of_targeted_receiver: Ruta del receptor objetivo
      - play_action: Si hubo play action (binary)
      - dropback_type: Tipo de dropback del QB
      - dropback_distance: Distancia que retrocedió el QB (yardas)
      - pass_location_type: Ubicación del QB al lanzar (Inside Tackle Box, Outside Left/Right)

  * Variables defensivas:

      - defenders_in_the_box: Número de defensas cerca de la LOS
      - team_coverage_man_zone: Tipo de cobertura (Man/Zone)
      - team_coverage_type: Tipo específico de cobertura

  * Variables de resultado:

      - play_nullified_by_penalty: Si la jugada fue anulada por penalti (Y/N)
      - penalty_yards: Yardas ganadas por penalti
      - pre_penalty_yards_gained: Yardas ganadas antes de penaltis
      - yards_gained: Yardas netas ganadas
      - expected_points: Puntos esperados en la jugada
      - expected_points_added: Delta de puntos esperados
      - pre_snap_home_team_win_probability: Win probability del equipo local antes del snap
      - pre_snap_visitor_team_win_probability: Win probability del visitante antes del snap
      - home_team_win_probability_added: Delta de win probability para local
      - visitor_team_win_probility_added: Delta de win probability para visitante
